<a href="https://colab.research.google.com/github/halfbug/colab/blob/main/retriever-vector-db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **LangChain and OpenAI installation**


In [4]:
try:
  import openai
  from langchain_openai import ChatOpenAI
except:
    !pip install openai
    !pip install langchain
    !pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.8 MB/s eta 0:00:00


In [5]:
import openai
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [6]:
from langchain.chat_models import ChatOpenAI
import datetime
current_date = datetime.datetime.now().date()
# llm_name = "gpt-4"
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.invoke("Hello world!")

gpt-3.5-turbo


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

# Scraper

In [7]:
import re
import requests
from bs4 import BeautifulSoup

def extract_availability(url):
    # Function to scrape the webpage for availability information
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
         # Search for specific keywords like "stock availability" on the webpage

        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # Search for either "stock" or "stock availability" in the text content
            # print(text_content)
            #  availability_pattern = re.compile(r"(?:stock|stock|availability|Number\sof\sItems\s*(\d+)|in\s*stock|(\w+)\s*stock\s*(\w+))", re.IGNORECASE)

            availability_pattern = re.compile(r"(?:stock|stock|availability|in\s*stock)", re.IGNORECASE)
            match = re.search(availability_pattern, text_content)
            # print(match.groups())
            if match:
                # print(match.group(0))
                return f"Availability: {match.group(0).strip()}"
        # If specific keywords not found, return "Scraped Availability: Unknown"
        return "Availability: Unknown"
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return "Availability: Unknown"

In [35]:
def scrape_text(url):
    # Function to scrape the webpage for availability information
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
         # Search for specific keywords like "stock availability" on the webpage

         # Step 3: Extract metadata
        meta_tags = soup.find_all('meta')
        metadata = {}
        for tag in meta_tags:
            if 'name' in tag.attrs:
                name = tag.attrs['name']
                content = tag.attrs.get('content', '')
                metadata[name] = content
            elif 'property' in tag.attrs:  # For OpenGraph metadata
                property = tag.attrs['property']
                content = tag.attrs.get('content', '')
                metadata[property] = content

        # Display the metadata
        # for key, value in metadata.items():
        #     print(f"{key}: {value}")

        # print( metadata)

        meta_tag = extract_product_info([metadata])

            # Find all link elements with the 'as' attribute set to 'image'
        image_links = soup.find_all('link', {'rel': 'preload', 'as': 'image'})

        # Extract and print the href attributes
        images = ''
        for link in image_links:
            image_url = link.get('href')
            if image_url:
                if image_url.startswith("//"):
                  # Add protocol (http assumed):
                  image_url= f"http:{image_url}"
                images = images +","+ image_url
                # print(image_url)

        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # Search for either "stock" or "stock availability" in the text content
            # print(text_content)
            return text_content + meta_tag + "Images : " + images
        return f"Content: Unknown on {url}"
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return f"Content: Unknown on {url}"

In [37]:
def google_scraper(query) :
    try:
        response = requests.get("https://www.google.com/search?q="+query.lower().replace(' ', '+ upc'))
        soup = BeautifulSoup(response.content, 'html.parser')

        # find first five link's content
        anchor_tags = soup.find_all('a', href=True)
        hrefs = []
        content = ""
        for tag in anchor_tags:
          href = tag.get('href')
          if href and href.startswith("/url?q=https") and "google" not in href.lower():  # Check if href exists (it might be None)
            parts = href.split('&', 1)
            parts = parts[0].split('/url?q=',1)
            # print(parts)
            hrefs.append(parts[1])
            scraped_content = scrape_text(parts[1])
            content += scraped_content + "\n ***"
            # print(scraped_content)
        # print(hrefs[:5])



        body_content = soup.find('body')  # Find the body tag

        if body_content:
           # Extract only the text content from the body tag
            text_content = body_content.get_text(strip=True)
            # print("\033[34m" + text_content )
            return text_content + content
    except Exception as e:
        print(f"Error occurred while scraping: {e}")
        return "No search result found: Unknown"
google_scraper("Apotheke 3 wick candles collection")

Skip to contentEnjoy complementary shipping on orders $100+ *Exclusions applyMenuCandles & DiffusersCandlesBest Selling FragranceClassic Candles2-Wick Ceramic Candles3-Wick CandlesFragrance BundlesDiscovery SetsMini CandlesTaper CandlesCandle AccessoriesDiffusersReed DiffusersRoom SpraysPura Smart Home DiffuserFragrance CardsSubscriptionsClassic Mystery Box SubscriptionPremium Mystery Box SubscriptionReed Diffuser RefillsFeaturedGift CardsAPOTHEKE x League Creative Co.APOTHEKE x MASTAPOTHEKE x SanctuaryBody CareBar SoapLiquid SoapLotionBody Care BundlesLiquid Soap RefillsFragrancesThe Signature CollectionApricot Red CurrantCanvasEarl Grey BittersHinoki LavenderSantal Rock RoseSea Salt GrapefruitTonka OakWild Mint and IvyWhite VetiverThe Charcoal Fragrance CollectionCharcoalCharcoal RougeThe Eden CollectionBlack Iris OakCedarwood GingerOrange Blossom NeroliSaffron VanillaFragrance FamiliesAromaticCitrusFloralFruityGourmandGreenWoodyAboutBlogCandle Buying GuideOur StoryVisit UsFind Your 

'Google×Please clickhereif you are not redirected within a few seconds.AllShoppingImagesVideosMapsNewsBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimShowing results forapotheke upc3upc wickupc candlesupc collectionSearch instead forapotheke upc3 upcwick upccandles upccollection3-Wick Candles | APOTHEKEapothekeco.com › collections › 3-wick-candlesMade with a premium soy wax blend and essential and perfume-grade fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Missing:upc3upcCanvas 3-Wick Candle - Apothekeapothekeco.com › products › canvas-3-wick-candleRating5.0(8)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a luxe white-translucent glass vessel. The\xa0...Missing:upc3upcApotheke 3-Wick Candles Collection - eBaywww.ebay.com › Home & Garden › Candles & Home Fragrance › Candles$106.00Apotheke 3-

In [10]:
import re

def extract_product_info(meta_tags):
    product_info = ""
    for meta_tag in meta_tags:
        if 'og:title' in meta_tag:
            product_info += f"\n ***Title: {meta_tag['og:title']}\n"
        if 'og:description' in meta_tag:
            product_info += f"Description: {meta_tag['og:description']}\n"
        if 'og:price:amount' in meta_tag and 'og:price:currency' in meta_tag:
            product_info += f"Price: {meta_tag['og:price:amount']} {meta_tag['og:price:currency']}\n"
        if 'og:image' in meta_tag:
                if len(meta_tag['og:image']) <= 200:  # Check if URL length is less than or equal to 200
                    product_info += f"Image URL: {meta_tag['og:image']} \n"
        if 'og:url' in meta_tag:
            product_info += f"URL: {meta_tag['og:url']} \n"
            # Call function to extract availability information
            availability_info = extract_availability(meta_tag['og:url'])
            product_info += f"{availability_info}\n"
            # Extract ASIN from URL if it's an Amazon link
            asin_match = re.search(r'/dp/(\w+)', meta_tag['og:url'])
            if asin_match:
                product_info += f"ASIN: {asin_match.group(1)} \n"
        if 'og:site_name' in meta_tag:
            product_info += f"Site Name: {meta_tag['og:site_name']} \n"
        if 'og:availability' in meta_tag:
            product_info += f"Availability: {meta_tag['og:availability']} \n"
        if 'og:type' in meta_tag:
                product_info += f"type: {meta_tag['og:type']} \n"
        else:
          product_info += f"type: text \n"
    return product_info

# Example usage:
# meta_tags = [{'apple-itunes-app': 'app-id=529356019, app-argument=https://www.crateandbarrel.com/apotheke-canvas-scented-3-wick-candle/s569504',
#  'og:image': 'https://cb.scene7.com/is/image/Crate/ApothekeCnvs3WickCandleSSF23', 'og:type': 'product', 'og:image:alt': 'Apotheke Canvas-Scented 3-Wick Candle',
#   'twitter:card': 'summary_large_image', 'twitter:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:site_name': 'Crate & Barrel', 'og:availability':
#    'InStock', 'og:price:amount': '$78.00', 'author': 'Crate & Barrel', 'og:title': 'Apotheke Canvas-Scented 3-Wick Candle + Reviews | Crate & Barrel', 'og:price:currency': 'USD', 'language':
#     'English', 'og:description': 'Shop Apotheke Canvas-Scented 3-Wick Candle.  This hand-poured candle fills the room with clean, refreshing scent.  Infused with essences of crisp linen, white musk,
#      lily of the valley and dew drop, the soy-blend wax candle burns for many hours releasing a sweet, fresh fragrance.', 'og:price:standard_amount': '$78.00', 'twitter:site': '@CrateandBarrel',
#      'viewport': 'width=device-width, initial-scale=1.0', 'fb:admins': '1769841725', 'og:url': 'https://www.amazon.com/dp/B09RTPD6ZN/ref=tsm_1_fb_lk'}]
# product_info = extract_product_info(meta_tags)
# print(product_info)

# Google Search API

In [11]:
from googleapiclient.discovery import build

API_KEY = userdata.get('GOOGLE_SEARCH_API')
# os.environ["GOOGLE_CSE_ID"] = userdata.get('GOOGLE_CSE_ID')

service = build("customsearch", "v1", developerKey=API_KEY)
# clientProd = build("shopping", "v1", developerKey=API_KEY)

In [19]:
def search_google(query):
    """
    Performs a custom search  using the Google Custom Search API.

    Args:
        query (str): The search query to use.
        num_results (int, optional): The number of image search results to return (default: 3).

    Returns:
        str: A string containing information about the image search results. Each result includes the image URL and its corresponding link.
    """
    # user_query= extract_text_between_hashes(query)
    results = service.cse().list(
        q=query, cx=userdata.get('GOOGLE_CSE_ID'),  # Replace with your custom search engine ID
        # searchType="image",  # Specify image search
        num=3
    ).execute()

    search_results = []
    for item in results.get("items", []):
        for key, value in item.items():
          print("\033[32m" + f"{key}: {value}" + "\033[0m")
        image_url = item["pagemap"]["cse_image"][0]["src"]
        print(item["pagemap"]["metatags"])
        search_results.append({"link":item["link"] ,
                              "title": item["title"] ,
                              "snippets":item["snippet"],
                              "image": image_url,
                              "meta": extract_product_info(item["pagemap"]["metatags"])})
        # search_results += extract_product_info(item["pagemap"]["metatags"])
        # search_results += f"image2: {image_url} \n"
        # search_results += f"link: {item['link']}"

    len(search_results)
    return search_results

# Example usage:
image_results = search_google("Apotheke 3 wick candles collection price images")
# image_results
print(image_results)

kind: customsearch#result
title: Apotheke 3-Wick Candles Collection : Health ... - Amazon.com
htmlTitle: Apotheke 3-Wick Candles Collection : Health ... - Amazon.com
link: https://www.amazon.com/Scented-Candles-Home-Soy-Wax-Jar-Candle/dp/B07PMY9PK6
displayLink: www.amazon.com
snippet: Captivating Fragrance: Transform any living space with the unforgettable scent of Charcoal 3-Wick Candle from APOTHEKE. Our candles are hand poured and made ...
htmlSnippet: Captivating Fragrance: Transform any living space with the unforgettable scent of Charcoal <b>3</b>-<b>Wick Candle</b> from <b>APOTHEKE</b>. Our <b>candles</b> are hand poured and made&nbsp;...
formattedUrl: https://www.amazon.com/Scented-Candles-Home-Soy.../B07PMY9PK6
htmlFormattedUrl: https://www.amazon.com/Scented-<b>Candles</b>-Home-Soy.../B07PMY9PK6
pagemap: {'cse_thumbnail': [{'src': 'https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSn_XK9lxk8YHeoZ-1cwFl6vAw52HIMW2FN2RbGuGT2KqCwycqvSOYtY-o', 'width': '238', 'height': '212'}

In [13]:
def combined_scraped_api_search(query):
  scraped_results = google_scraper(query)
  search_api_results = search_google(query)
  return f"""{scraped_results} \n {search_api_results} """


# RAG

In [40]:
column =  "Apotheke 3 wick candles collection"

## Document Splitting

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)


In [41]:
splits = text_splitter.split_text(google_scraper(column))


Skip to contentEnjoy complementary shipping on orders $100+ *Exclusions applyMenuCandles & DiffusersCandlesBest Selling FragranceClassic Candles2-Wick Ceramic Candles3-Wick CandlesFragrance BundlesDiscovery SetsMini CandlesTaper CandlesCandle AccessoriesDiffusersReed DiffusersRoom SpraysPura Smart Home DiffuserFragrance CardsSubscriptionsClassic Mystery Box SubscriptionPremium Mystery Box SubscriptionReed Diffuser RefillsFeaturedGift CardsAPOTHEKE x League Creative Co.APOTHEKE x MASTAPOTHEKE x SanctuaryBody CareBar SoapLiquid SoapLotionBody Care BundlesLiquid Soap RefillsFragrancesThe Signature CollectionApricot Red CurrantCanvasEarl Grey BittersHinoki LavenderSantal Rock RoseSea Salt GrapefruitTonka OakWild Mint and IvyWhite VetiverThe Charcoal Fragrance CollectionCharcoalCharcoal RougeThe Eden CollectionBlack Iris OakCedarwood GingerOrange Blossom NeroliSaffron VanillaFragrance FamiliesAromaticCitrusFloralFruityGourmandGreenWoodyAboutBlogCandle Buying GuideOur StoryVisit UsFind Your 

In [42]:
splits


['Google×Please clickhereif you are not redirected within a few seconds.AllShoppingImagesVideosMapsNewsBooksSearch toolsAny timeAny timePast hourPast 24 hoursPast weekPast monthPast yearAll resultsAll resultsVerbatimShowing results forapotheke upc3upc wickupc candlesupc collectionSearch instead forapotheke upc3 upcwick upccandles upccollection3-Wick Candles | APOTHEKEapothekeco.com › collections › 3-wick-candlesMade with a premium soy wax blend and essential and perfume-grade fragrance oils, our luxury scented 3-wick candles are designed to provide a clean-burning\xa0...Missing:upc3upcCanvas 3-Wick Candle - Apothekeapothekeco.com › products › canvas-3-wick-candleRating5.0(8)·$78.00·In stockThis 3-Wick Scented Candle is made with a premium soy wax blend and essential and perfume-grade fragrance oils in a luxe white-translucent glass vessel. The\xa0...Missing:upc3upcApotheke 3-Wick Candles Collection - eBaywww.ebay.com › Home & Garden › Candles & Home Fragrance › Candles$106.00Apotheke 3

# LangChain Expression Language (LCEL)

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableMap

In [15]:
template = """
As a search analyst, your task is to find specific information online. We've provided context to assist:

{context}

Answer the following question based solely on the context:

Question: {question}

Provide the requested information in this format:

Brand name:
Product name:
Product image(s): [provide at least one link]
Color: [list colors, separated by commas]
Category: [identify the product's category on Amazon]
Description: [include a brief product description]
Sold on Amazon: true / false
Amazon price: [indicate the price range]
ASIN: [list ASIN codes, separated by commas]
UPC: [mention the UPC]

If the information isn't available, conduct your own search and provide an answer.
"""
prompt = ChatPromptTemplate.from_template(template)

In [16]:
model = llm
output_parser = StrOutputParser()



In [17]:
chain = RunnableMap({
    "context": lambda x: google_scraper(x["question"]),     #combined_scraped_api_search(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [18]:
print(chain.invoke({"question": "Apotheke 3-Wick Candles Collection charcoal"}))

{'viewport': 'width=device-width,initial-scale=1.0', 'description': 'Our number 1 best seller, Charcoal, featured in our signature matte black vessel will elevate any room in your home. Notes of cedarwood and sandalwood blended with smokey amber and oud come together to create a broody full-bodied fragrance.', 'theme-color': '#2f2d2e', 'og:site_name': 'APOTHEKE', 'og:url': 'https://apothekeco.com/products/charcoal-3-wick-candle', 'og:title': 'Charcoal 3-Wick Candle', 'og:type': 'product', 'og:description': 'Our number 1 best seller, Charcoal, featured in our signature matte black vessel will elevate any room in your home. Notes of cedarwood and sandalwood blended with smokey amber and oud come together to create a broody full-bodied fragrance.', 'og:image': 'http:products/AP03-CH_Charcoal_26oz_3WickCandle.jpg', 'og:image:secure_url': 'https:products/AP03-CH_Charcoal_26oz_3WickCandle.jpg', 'og:image:width': '1000', 'og:image:height': '1000', 'og:price:amount': '78.00', 'og:price:currenc

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 16576 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}